In [1]:
from pyomo.environ import *
import os
import idaes
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
from idaes.core.util.math import safe_log
from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
# Parameters
component_list = ["benzene","toluene"]
phase_list = ["Liq", "Vap"]
pressure_crit_data = {"benzene": 48.9e5, "toluene": 41.0e5}
temperature_crit_data = {"benzene": 562.2, "toluene": 591.8}
kappa_data = {
            ("benzene", "benzene"): 0.0000,
            ("benzene", "toluene"): 0.0000,
            ("toluene", "benzene"): 0.0000,
            ("toluene", "toluene"): 0.0000,
        }
omega_data = {"benzene": 0.212, "toluene": 0.263}
R = 8.314462618

In [3]:
m = ConcreteModel()
# define external function for finding cubic roots
m.crl = ExternalFunction(
    library=os.path.join(idaes.bin_directory, "cubic_roots.so"),
    function="cubic_root_l_ext",
)
m.crh = ExternalFunction(
    library=os.path.join(idaes.bin_directory, "cubic_roots.so"),
    function="cubic_root_h_ext",
)

In [4]:
m.Comp = Set(initialize=component_list)
m.Phase = Set(initialize=phase_list)

m.flow_mol = Var(initialize=1.0,domain=NonNegativeReals) # F

m.mole_frac_comp = Var(m.Comp, bounds=(0, None), initialize=0.5) # z

m.flow_mol_phase = Var(m.Phase, initialize=0.5, domain=NonNegativeReals) # L, V

m.mole_frac_phase_comp = Var(m.Phase, m.Comp, initialize=0.5,bounds=(0, None)) # x, y

def rule_total_mass_balance(m): # L + V = F
    return m.flow_mol_phase["Liq"] + m.flow_mol_phase["Vap"] == m.flow_mol
m.total_flow_balance = Constraint(rule=rule_total_mass_balance)

def rule_comp_mass_balance(m, c): # L*x + V*y = F*z
    return (
        m.flow_mol * m.mole_frac_comp[c]
        == m.flow_mol_phase["Liq"] * m.mole_frac_phase_comp["Liq", c]
        + m.flow_mol_phase["Vap"] * m.mole_frac_phase_comp["Vap", c]
    )
m.component_flow_balances = Constraint(m.Comp, rule=rule_comp_mass_balance)

def rule_mole_frac(m): # sum(x) - sum(y) = 1
    return (
        sum(m.mole_frac_phase_comp["Liq", c] for c in m.Comp)
        - sum(m.mole_frac_phase_comp["Vap", c] for c in m.Comp)
        == 0
    )
m.sum_mole_frac = Constraint(rule=rule_mole_frac)

m.T = Var(initialize = 298.15)
m.P = Var(initialize = 101325)

m.Tc = Param(m.Comp,initialize = temperature_crit_data)
m.Pc = Param(m.Comp,initialize = pressure_crit_data)
m.kappa = Param(m.Comp, m.Comp, initialize = kappa_data)
m.omega = Param(m.Comp, initialize = omega_data)

# for SRK cEOS
m.OmegaA = Param(default=0.45724)
m.OmegaB = Param(default=0.07780)
m.EoS_u = Param(default=2)
m.EoS_w = Param(default=-1)
m.EoS_p = sqrt(m.EoS_u**2 - 4 * m.EoS_w)

def func_fw(m,j):
    return 0.37464 + 1.54226 * m.omega[j] - 0.26992 * m.omega[j] ** 2
m.fw = Param(m.Comp, initialize = func_fw)

def func_a(m, j):
    return (m.OmegaA * ((R * m.Tc[j]) ** 2 / m.Pc[j])
        * ((1 + m.fw[j] * (1 - sqrt(m.T / m.Tc[j]))) #是不是平方？？？
            ** 2))
m.a = Expression(m.Comp,rule=func_a)

def func_b(m, j):
    return (
        m.OmegaB
        * R * m.Tc[j] / m.Pc[j]
    )
m.b = Param(m.Comp,initialize=func_b)

def rule_am(m, p):
    return sum(
    sum(
        m.mole_frac_phase_comp[p, i]
        * m.mole_frac_phase_comp[p, j]
        * sqrt(m.a[i] * m.a[j])
        * (1 - m.kappa[i, j])
        for j in m.Comp
    )
    for i in m.Comp
    )
m.am = Expression(m.Phase, rule=rule_am)

def rule_bm(m, p):
    return sum(m.mole_frac_phase_comp[p,i] * m.b[i] for i in m.Comp)
m.bm = Expression(m.Phase, rule=rule_bm)

def func_A(m,p):
    return m.am[p]*m.P/(R*m.T)**2
m.A = Expression(m.Phase, rule=func_A)

def func_B(m,p):
    return m.bm[p]*m.P/(R*m.T)
m.B = Expression(m.Phase, rule=func_B)

In [5]:
# 定义三次方程一般形式的系数
def rule_cubic_coef_b(m,p):
    return (-(1+m.B[p]-m.EoS_u*m.B[p]))
m.cubic_coef_b = Expression(m.Phase,rule=rule_cubic_coef_b)

def rule_cubic_coef_c(m,p):
    return (m.A[p]-m.EoS_u*m.B[p]-(m.EoS_u-m.EoS_w)*m.B[p]**2)
m.cubic_coef_c = Expression(m.Phase,rule=rule_cubic_coef_c)

def rule_cubic_coef_d(m,p):
    return (-m.A[p]*m.B[p]-m.EoS_w*m.B[p]**2-m.EoS_w*m.B[p]**3)
m.cubic_coef_d = Expression(m.Phase,rule=rule_cubic_coef_d)

def evaluate_root(m,p):
    if p == 'Vap':
        return m.crh.evaluate(args=(m.cubic_coef_b[p], m.cubic_coef_c[p], m.cubic_coef_d[p]))
    else:
        return m.crl.evaluate(args=(m.cubic_coef_b[p], m.cubic_coef_c[p], m.cubic_coef_d[p]))
m.external_expr = Expression(m.Phase, rule=evaluate_root)

m.Z = Var(m.Phase, initialize = 1)
def rule_cubic_Z(m,p):
    return m.Z[p] == m.external_expr[p]
m.cubic_z = Constraint(m.Phase, rule = rule_cubic_Z)

In [6]:
# 用压缩因子计算相平衡
def rule_delta(m, p, i):
    # See pg. 145 in Properties of Gases and Liquids
    return (2 * sqrt(m.a[i]) / m.am[p]
        * sum(
            m.mole_frac_phase_comp[p, j]
            * sqrt(m.a[j])
            * (1 - m.kappa[i, j])
            for j in m.Comp
        )
    )
m.delta = Expression(m.Phase, m.Comp, rule=rule_delta)

def ln_fug_coeff_cubic(b, p, j):
    # See pg. 145 in Properties of Gases and Liquids
    return (
            b.b[j] / b.bm[p] * (b.Z[p] - 1) * (b.B[p] * b.EoS_p)
            - safe_log(b.Z[p] - b.B[p], eps=1e-6)
            * (m.B[p] * m.EoS_p)
            + b.A[p]
            * (b.b[j] / b.bm[p] - b.delta[p, j])
            * safe_log(
                (2 * b.Z[p] + b.B[p] * (b.EoS_u + b.EoS_p))
                / (2 * b.Z[p] + b.B[p] * (b.EoS_u - b.EoS_p)),
                eps=1e-6,
            )
        ) / (b.B[p] * b.EoS_p) + log(b.mole_frac_phase_comp[p, j])
m.ln_fug_coeff = Expression(m.Phase, m.Comp, rule = ln_fug_coeff_cubic)

In [7]:
# 相平衡约束
def rule_VLE(m,c):
    return m.ln_fug_coeff['Vap',c] == m.ln_fug_coeff['Liq',c]
m.rule_equilibrium = Constraint(m.Comp, rule = rule_VLE)

In [8]:
m.flow_mol.fix(100)
m.mole_frac_comp["benzene"].fix(0.5)
m.mole_frac_comp["toluene"].fix(0.5)
m.T.fix(368)
m.P.fix(1e5)

In [9]:
degrees_of_freedom(m)

0

In [10]:
solver = SolverFactory('ipopt')
solver.options['tol'] = 1e-6
results = solver.solve(m,tee=True)

Ipopt 3.13.2: tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. S

In [11]:
for p in m.Phase:
    for c in m.Comp:
        print(value(m.Z[p]))

0.003983679341526769
0.003983679341526769
0.9441927576897298
0.9441927576897298


In [12]:
m.flow_mol_phase.pprint()

flow_mol_phase : Size=2, Index=Phase
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
    Liq :     0 : 13.606764528661463 :  None : False : False : NonNegativeReals
    Vap :     0 :  86.39323547133854 :  None : False : False : NonNegativeReals


In [13]:
m.mole_frac_phase_comp.pprint()

mole_frac_phase_comp : Size=4, Index=mole_frac_phase_comp_index
    Key                : Lower : Value              : Upper : Fixed : Stale : Domain
    ('Liq', 'benzene') :     0 : 0.3323159828713967 :  None : False : False :  Reals
    ('Liq', 'toluene') :     0 : 0.6676840171286033 :  None : False : False :  Reals
    ('Vap', 'benzene') :     0 : 0.5264099026253737 :  None : False : False :  Reals
    ('Vap', 'toluene') :     0 : 0.4735900973746263 :  None : False : False :  Reals


In [14]:
m.mole_frac_comp.pprint()

mole_frac_comp : Size=2, Index=Comp
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
    benzene :     0 :   0.5 :  None :  True :  True :  Reals
    toluene :     0 :   0.5 :  None :  True :  True :  Reals
